In [88]:
# Mounting Google Drive
from google.colab import drive
from os.path import join
ROOT = '/content/drive'
print(ROOT)
drive.mount(ROOT)

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
%pwd

'/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition'

In [90]:
%cd /content/drive/My\ Drive/Colab Notebooks/ProjectData

/content/drive/My Drive/Colab Notebooks/ProjectData


## Importing the dataset and necessary libraries

In [91]:
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
%matplotlib inline

In [92]:
Dataset_path = "FaceRecognition/dataset"
face_detection_path= "FaceRecognition/face_detection_model/res10_300x300_ssd_iter_140000.caffemodel"
proto_path = "FaceRecognition/face_detection_model/deploy.prototxt"
embedding_gen_path = "FaceRecognition/openface_nn4.small2.v1.t7"
output_path = "FaceRecognition/output"

In [93]:
print("Loading face detector.....")
detector = cv2.dnn.readNetFromCaffe(proto_path, face_detection_path)

Loading face detector.....


In [94]:
print("Loading face embedder....")
embedder = cv2.dnn.readNetFromTorch(embedding_gen_path);

Loading face embedder....


In [95]:
imagepath = list(paths.list_images(Dataset_path))


In [96]:
imagepath

['FaceRecognition/dataset/arpan/IMG-20180811-WA0022.jpg',
 'FaceRecognition/dataset/arpan/IMG-20200520-WA0004.jpg',
 'FaceRecognition/dataset/arpan/IMG-20200902-WA0007.jpg',
 'FaceRecognition/dataset/arpan/IMG-20180925-WA0019.jpg',
 'FaceRecognition/dataset/arpan/IMG-20180104-WA0004.jpg',
 'FaceRecognition/dataset/arpan/IMG-20180114-WA0040.jpg',
 'FaceRecognition/dataset/arpan/IMG-20200307-WA0016.jpg',
 'FaceRecognition/dataset/arpan/IMG-20200902-WA0015.jpg',
 'FaceRecognition/dataset/arpan/arpan1.jpg',
 'FaceRecognition/dataset/arpan/IMG_20190210_155413235.jpg',
 'FaceRecognition/dataset/arpan/IMG-20180208-WA0000.jpg',
 'FaceRecognition/dataset/arpan/IMG_20181224_121707408_HDR.jpg',
 'FaceRecognition/dataset/remon/IMG-20191028-WA0035.jpg',
 'FaceRecognition/dataset/remon/IMG-20191228-WA0018.jpg',
 'FaceRecognition/dataset/remon/IMG-20200102-WA0005.jpg',
 'FaceRecognition/dataset/remon/IMG-20190803-WA0002.jpg',
 'FaceRecognition/dataset/remon/IMG-20181027-WA0016.jpg',
 'FaceRecognition

In [97]:
knownEmbeddings = []
knownNames = []
total = 0
for (i, imageP) in enumerate(imagepath):
  print("Processing image ",int(i+1),"/",len(imagepath)-1)
  name = imageP.split(os.path.sep)[-2]

  image =  cv2.imread(imageP)
  image = imutils.resize(image, width=600)
  (h,w) = image.shape[:2]
  '''if(i % 5 == 0):
    plt.imshow(image)
    plt.show()
  '''
  blob = cv2.dnn.blobFromImage(cv2.resize(image, (300,300)),1.0, (300,300),(104.0, 177.0, 123.0), swapRB= False, crop = False)
  detector.setInput(blob)
  detections = detector.forward()

  for i in range(0,detections.shape[2]):
    confidence = detections[0,0,i,2];

    if confidence < 0.5:
      continue

    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
    (startX, startY, endX, endY) = box.astype("int")


    cv2.rectangle(image, (startX, startY), (endX, endY), (0,0,255), 2)

    #text = "Confidence: ",confidence
    #image = cv2.putText(image,text,(startX, startY), cv2.FONT_HERSHEY_SIMPLEX , 0.45, (0,0,255),2) 
    face = image[startY:endY, startX:endX]
    (fH, fW) = face.shape[:2]

    if fH < 20 or fW < 20:
      continue

    faceBlob = cv2.dnn.blobFromImage(face, 1/255, (96,96), (0,0,0), swapRB = True, crop= False)

    embedder.setInput(faceBlob)
    vec = embedder.forward()
    knownNames.append(name)
    knownEmbeddings.append(vec.flatten())

    total = total+ 1



  cv2_imshow(image)



In [98]:
print("Serialising",total, "images and saving it into pickle files")
data = {"name":knownNames, "embedding":knownEmbeddings}
pickcle_path = output_path+"/data.pickle"
f = open(pickcle_path, "wb")
f.write(pickle.dumps(data))
f.close()

Serialising 48 images and saving it into pickle files


In [99]:
image1 = cv2.imread("FaceRecognition/dataset/arpan/arpan1.jpg")
image1.shape

(3120, 4160, 3)

In [100]:
detections[0,0,10]

array([0.        , 1.        , 0.08736235, 0.4320156 , 0.32942814,
       0.5610454 , 0.51926637], dtype=float32)

In [101]:
knownNames

['arpan',
 'arpan',
 'arpan',
 'arpan',
 'arpan',
 'arpan',
 'arpan',
 'arpan',
 'arpan',
 'arpan',
 'arpan',
 'arpan',
 'remon',
 'remon',
 'remon',
 'remon',
 'remon',
 'remon',
 'remon',
 'remon',
 'remon',
 'remon',
 'remon',
 'remon',
 'daw',
 'daw',
 'daw',
 'daw',
 'daw',
 'daw',
 'daw',
 'daw',
 'daw',
 'daw',
 'daw',
 'daw',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown']

In [102]:
!pwd

/content/drive/My Drive/Colab Notebooks/ProjectData


In [103]:
%cd FaceRecognition/

/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition


In [104]:
!git status

On branch master
Your branch is ahead of 'origin/master' by 5 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   ImageDetection.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [105]:
!git add .


In [106]:
!git config --global user.email "suranjandaw@gmail.com"
!git config --global user.name "SuranjanDaw"

In [107]:
!git commit -m"More data added"

[master 68fae45] More data added
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite ImageDetection.ipynb (71%)


In [108]:
!git remote -v

origin	https://github.com/SuranjanDaw/FaceRecognition (fetch)
origin	https://github.com/SuranjanDaw/FaceRecognition (push)


In [109]:
!git push origin master

fatal: could not read Username for 'https://github.com': No such device or address


In [110]:
!git pull origin master

fatal: could not read Username for 'https://github.com': No such device or address


In [111]:
!git status

On branch master
Your branch is ahead of 'origin/master' by 6 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   ImageDetection.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [112]:
!git commit -m"merging pull"

On branch master
Your branch is ahead of 'origin/master' by 6 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
	modified:   ImageDetection.ipynb

no changes added to commit


In [113]:
!git log --graph --oneline

* 68fae45 (HEAD -> master) More data added
* 1fcdbf0 More data added
* 5f866fc More data added
* 3b05f78 More data added
* 620ffbf More data added
* cec2c3f More data added
* bea8850 (origin/master, origin/HEAD) Updated DataSetup and Env_setup
* 785e98a Added untracked ipython/colab notebooks
* 156905b The dataset is trained with new data. Results: Confidence score increased
* f86d358 More data added
* 85fb770 Image recognition integrated
*   297ba50 merging pull
|\  
| * b7d02dc Importing DataSet from git to google drive and setting project structure
* | 01542f3 Image detection integrated with 128D embedding generated
|/  
* f848202 output and image folder added with test image
* dfae9ee face_detection_model (caffe)added and 128-D embedder added
* b7bfb1c dataset added
* 6cbdb5c Environment Setup done
* 8b21f8c Created using Colaboratory
* 75cde1d Created using Colaboratory
* 96772a2 Create README.md


In [114]:
!git push origin master

fatal: could not read Username for 'https://github.com': No such device or address
